In [1]:
"""
Task 3: Model 2 - The Offline Reinforcement Learning Agent

Loads the preprocessed data from Task 1.
Frames the problem as a one-step MDP (Contextual Bandit).
Engineers rewards based on financial outcomes (P&L).
Trains an offline RL agent (Discrete CQL) using d3rlpy.

Saves the following files:
- models/cql_agent.d3
- models/reward_scaler.pkl
"""

'\nTask 3: Model 2 - The Offline Reinforcement Learning Agent\n\nLoads the preprocessed data from Task 1.\nFrames the problem as a one-step MDP (Contextual Bandit).\nEngineers rewards based on financial outcomes (P&L).\nTrains an offline RL agent (Discrete CQL) using d3rlpy.\n\nSaves the following files:\n- models/cql_agent.d3\n- models/reward_scaler.pkl\n'

In [3]:
!pip install d3rlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 11.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.1/201.1 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827726 sha256=8f00d7970144e5c9b23436f70a1a9c345084e05216fea8a3bd5f350a26b19153
  Stored in directory: /root/.cache/pip/wheels/95/51/6c/9bb05ebbe7c5cb8171dfaa3611f32622ca4658d53f31c79077
Successfully built gym
  Attempting uninstall: gymnasium
    Found existing installation: gymnasium 1.2.1
    Uninstalling gymnasium-1.2.1:
      Successfully uninstalled gymnasium-1.2.1
  Attempting uninstall: gy

In [5]:
import numpy as np
import pandas as pd
import d3rlpy
from d3rlpy.dataset import MDPDataset
from sklearn.preprocessing import StandardScaler
import joblib
import os

# --- Configuration ---
RANDOM_SEED = 42
DATA_DIR = 'data'
MODEL_DIR = 'models'

# Set Random Seed
np.random.seed(RANDOM_SEED)

In [7]:
def main():

    # --- Load Preprocessed Data ---
    print(f"Loading data from '{DATA_DIR}'...")
    try:
        X_train_final = joblib.load(os.path.join(DATA_DIR, 'X_train_final.pkl'))
        df_model = joblib.load(os.path.join(DATA_DIR, 'df_model_for_rewards.pkl'))
        train_indices = joblib.load(os.path.join(DATA_DIR, 'train_indices.pkl'))
    except FileNotFoundError:
        print("Error: Processed data files not found.")
        print("Please run `task_1_preprocessing.py` first.")
        return

    # Create model directory if it doesn't exist
    if not os.path.exists(MODEL_DIR):
        os.makedirs(MODEL_DIR)
        print(f"Created directory: {MODEL_DIR}")

    # ---  Engineer RL Dataset Components ---
    print("Step 1: Engineering RL dataset (States, Actions, Rewards, Terminals)...")

    # States (s): Preprocessed features from Task 1
    observations_train = X_train_final.to_numpy()

    # Actions (a): In this dataset of *accepted* loans, the observed action was always Approve (1).
    actions_train = np.ones(len(observations_train), dtype=np.int32)

    # Rewards (R): Calculate Profit/Loss based on actual outcomes.

    print("Calculating raw rewards...")
    rewards_train_raw = np.where(
        df_model.loc[train_indices, 'is_default'] == 0, # Fully Paid
        df_model.loc[train_indices, 'total_rec_int'],    # Profit = interest received
        df_model.loc[train_indices, 'total_pymnt'] - df_model.loc[train_indices, 'loan_amnt'] # Loss
    )
    print(f"Sample raw rewards (Train): {rewards_train_raw[:5]}")

    # Reward Scaling (Important for Q-learning stability)
    reward_scaler = StandardScaler()
    rewards_train_scaled = reward_scaler.fit_transform(rewards_train_raw.reshape(-1, 1)).flatten()
    print(f"Mean scaled reward (Train): {np.mean(rewards_train_scaled):.2f}")

    # Terminals (d): This is a one-step decision problem (contextual bandit).
    terminals_train = np.ones(len(observations_train), dtype=np.float32)

    print("RL dataset components assembled.")

    # --- Create d3rlpy Dataset ---
    print("Step 2: Building d3rlpy MDPDataset...")
    # Note: This is a one-step MDP.
    dataset = MDPDataset(
        observations=observations_train,
        actions=actions_train,
        rewards=rewards_train_scaled,
        terminals=terminals_train
    )
    print("MDPDataset created.")

    # --- Configure and Train CQL Agent ---
    print("Step 3: Configuring and training the Discrete CQL agent...")

    # Configure CQL for a discrete action space {0, 1}
    cql_config = d3rlpy.algos.DiscreteCQLConfig(
        batch_size=128,
        learning_rate=6.25e-5,
        alpha=1.0,
    )

    cql_agent = cql_config.create(device=False) # Use CPU

    print("Training CQL agent...")
    cql_agent.fit(
        dataset,
        n_steps=50000, # Can increase for better convergence
        n_steps_per_epoch=5000
    )
    print("--- CQL Training Complete ---")

    # --- Save Agent and Reward Scaler ---
    agent_path = os.path.join(MODEL_DIR, 'cql_agent.d3')
    scaler_path = os.path.join(MODEL_DIR, 'reward_scaler.pkl')

    cql_agent.save_model(agent_path)
    joblib.dump(reward_scaler, scaler_path)

    print(f"CQL agent saved to {agent_path}")
    print(f"Reward scaler saved to {scaler_path}")
    print("--- Task 3: CQL Training Complete ---")

if __name__ == "__main__":
    main()


--- Task 3: Building and Training Offline RL Agent (CQL) ---
Loading data from 'data'...
Step 1: Engineering RL dataset (States, Actions, Rewards, Terminals)...
Calculating raw rewards...
Sample raw rewards (Train): [-3788.31  2392.37   373.43 -5368.02  4087.2 ]
Mean scaled reward (Train): -0.00
RL dataset components assembled.
Step 2: Building d3rlpy MDPDataset...
2025-10-30 07:51.59 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('int32')], shape=[(1,)]) observation_signature=Signature(dtype=[dtype('float64')], shape=[(132,)]) reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)])
2025-10-30 07:51.59 [info     ] Action-space has been automatically determined. action_space=<ActionSpace.DISCRETE: 2>
2025-10-30 07:52.01 [info     ] Action size has been automatically determined. action_size=2
MDPDataset created.
Step 3: Configuring and training the Discrete CQL agent...
Training CQL agent...
2025-10-30 07:52.02 [info     ] d

Epoch 1/10:   0%|          | 0/5000 [00:00<?, ?it/s]

2025-10-30 07:53.36 [info     ] DiscreteCQL_20251030075210: epoch=1 step=5000 epoch=1 metrics={'time_sample_batch': 0.004741993665695191, 'time_algorithm_update': 0.012149934673309326, 'loss': 0.2892695835068822, 'td_loss': 0.2765618439823389, 'conservative_loss': 0.012707739434438554, 'time_step': 0.017119435310363768} step=5000
2025-10-30 07:53.36 [info     ] Model parameters are saved to d3rlpy_logs/DiscreteCQL_20251030075210/model_5000.d3


Epoch 2/10:   0%|          | 0/5000 [00:00<?, ?it/s]

2025-10-30 07:55.03 [info     ] DiscreteCQL_20251030075210: epoch=2 step=10000 epoch=2 metrics={'time_sample_batch': 0.0048064826965332035, 'time_algorithm_update': 0.01205392894744873, 'loss': 0.2741426198735833, 'td_loss': 0.27412186073064804, 'conservative_loss': 2.0759493547029708e-05, 'time_step': 0.01708390989303589} step=10000
2025-10-30 07:55.03 [info     ] Model parameters are saved to d3rlpy_logs/DiscreteCQL_20251030075210/model_10000.d3


...

2025-10-30 08:03.47 [info     ] DiscreteCQL_20251030075210: epoch=8 step=40000 epoch=8 metrics={'time_sample_batch': 0.00482250714302063, 'time_algorithm_update': 0.012024013662338257, 'loss': 0.26945861524790526, 'td_loss': 0.26945861524790526, 'conservative_loss': 3.1664967536926268e-12, 'time_step': 0.01708188319206238} step=40000
2025-10-30 08:03.47 [info     ] Model parameters are saved to d3rlpy_logs/DiscreteCQL_20251030075210/model_40000.d3


Epoch 9/10:   0%|          | 0/5000 [00:00<?, ?it/s]

2025-10-30 08:05.15 [info     ] DiscreteCQL_20251030075210: epoch=9 step=45000 epoch=9 metrics={'time_sample_batch': 0.0048926291465759275, 'time_algorithm_update': 0.012241019344329834, 'loss': 0.2709443375915289, 'td_loss': 0.2709443375915289, 'conservative_loss': 2.9802322387695314e-12, 'time_step': 0.017358287143707275} step=45000
2025-10-30 08:05.15 [info     ] Model parameters are saved to d3rlpy_logs/DiscreteCQL_20251030075210/model_45000.d3


Epoch 10/10:   0%|          | 0/5000 [00:00<?, ?it/s]

2025-10-30 08:06.41 [info     ] DiscreteCQL_20251030075210: epoch=10 step=50000 epoch=10 metrics={'time_sample_batch': 0.004824909257888794, 'time_algorithm_update': 0.012061398315429687, 'loss': 0.26922877210974694, 'td_loss': 0.26922877210974694, 'conservative_loss': 2.7939677238464356e-12, 'time_step': 0.017117854404449462} step=50000
2025-10-30 08:06.41 [info     ] Model parameters are saved to d3rlpy_logs/DiscreteCQL_20251030075210/model_50000.d3
--- CQL Training Complete ---
CQL agent saved to models/cql_agent.d3
Reward scaler saved to models/reward_scaler.pkl
--- Task 3: CQL Training Complete ---
